# Проект по А/B-тестированию. 
# Исследование изменений, связанных с внедрением улучшенной рекомендательной системы


В наши задачи входило провести оценку результатов A/B-теста. Нам было предоставлено несколько датасетов: датасет с действиями пользователей, несколько вспомогательных датасетов и техническое задание.

Согласно ТЗ, в течение 14 дней происходило распределение пользователей по двум группам и затем по каждому пользователю собиралась информация о совершаемых им событиях в течение 14-дневного лайфтайма.

Ход исследования:

Загрузка данных и предварительное изучение переменных

Предобработка и анализ переменных

Исследовательский анализ данных: конверсия, построение воронок продаж

Анализ A/B-теста
 
Выводы

Загрузка библиотек

In [1]:
import pandas as pd
import matplotlib
import scipy
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm 
import seaborn as sns 
import pandas.tseries 
import plotly.express as px
import plotly.graph_objects as go
import warnings




from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import sem
from scipy.stats import kurtosis
from scipy.stats import skew
from statsmodels import robust
from scipy import stats
from scipy.stats import norm
from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
from statistics import median

Настройка среды

In [2]:
# вывод не более 3х знаков после запятой 
# pd.set_option('display.float_format', '{:.3f}'.format)

# вывод всех столбцов
pd.set_option('display.max_columns', None)

#вывод всех строк
#pd.set_option('display.max_rows', None)

#сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)


#warnings.filterwarnings('ignore')

## Загрузка данных и изучение общей информации

Загрузка функций для первичного анализа

In [7]:
def data_analysis(data):
    
    # Приведение названий столбцов к одному виду
    data.columns = data.columns.str.lower()
    
    # Вывод первых 5 строк датасета
    print("Первые 5 строк датасета:")
    display(data.head(5))
    print("----------------------------------------")

    # Проверка на полные дубликаты строк
    if data.duplicated().any():
        print("Найдены полные дубликаты строк в датасете.")
        num_duplicates = data.duplicated().sum()
        total_rows = len(data)
        duplicate_ratio = num_duplicates / total_rows
        print(f"Количество строк-дубликатов (кроме первых встреченных): {num_duplicates}")
        print(f"Доля дубликатов от общего числа строк: {duplicate_ratio:.2%}")
    else:
        print("Полных дубликатов строк в датасете не обнаружено.")
    print("----------------------------------------")

    

# Вызов функции для первичного анализа данных
# data_analysis(data)

In [8]:
def dataframe_statistics(data):
    '''
    dataframe statistics gathering function
    data — input dataframe
    returns table filled with statistics
    '''
    
    df_data = []
    df_cols = ['name', 'dtype', 'count', 'na_count', 'zero_count',
               'unique_count', 'neg_count', 'mean', 'se_mean', 'sigma', 'min', '25%', '50%',
               '75%', 'max', 'sum', 'kurtosis-3', 'adj_skewness', 'mean_-3sigma','mean_+3sigma']
    
    # iterating over columns
    for column_name in data.columns:
        column_type = data[column_name].dtype
        column_count = data[column_name].count()
        column_na_count = data[column_name].isna().sum()
        column_zero_count = (data[column_name] == 0).sum()
        column_unique_count = data[column_name].nunique()
        
        # checking for ints and floats
        if 'int' in str(column_type) or 'float' in str(column_type):
            column_negative_count = (data[column_name] < 0).sum()
            column_mean = round(data[column_name].mean(), 2)
            data_dropna=data.dropna(subset=[column_name])#удаление пропусков
            se_mean = round(sem(data_dropna[column_name]), 2)
            sigma = round(data[column_name].var(ddof=0)**0.5, 2)
            column_min = round(data[column_name].min(), 2)
            column_25 = round(data[column_name].quantile(0.25), 2)
            column_50 = round(data[column_name].quantile(0.50), 2)
            column_75 = round(data[column_name].quantile(0.75), 2)
            column_max = round(data[column_name].max(), 2)
            column_sum = round(data[column_name].sum(), 2)
            column_kurtosis = round(kurtosis(data_dropna[column_name], fisher=False)-3, 3)
            column_skewness= round(skew(data_dropna[column_name], bias=False), 3)
            mean_minus_three_sigmas = column_mean-3*sigma
            mean_plus_three_sigmas = column_mean+3*sigma
           
            
            
            
            # calculating outliers
            #outliers_left_count = data[data[column_name] < (column_mean - 3 * sigma)][column_name].count()
            #outliers_right_count = data[data[column_name] > (column_mean + 3 * sigma)][column_name].count()
        else:
            column_negative_count = np.nan
            column_mean = np.nan
            sigma = np.nan
            column_min = np.nan
            column_25 = np.nan
            column_50 = np.nan
            column_75 = np.nan
            column_max = np.nan
            column_sum = np.nan
            se_mean = np.nan
            column_kurtosis = np.nan
            column_skewness = np.nan
            mean_plus_three_sigmas = np.nan
            mean_minus_three_sigmas = np.nan
         
            #outliers_left_count = np.nan
            #outliers_right_count = np.nan
 
        # gathering results
        df_data.append([column_name, column_type, column_count, column_na_count,
                        column_zero_count, column_unique_count,
                        column_negative_count, column_mean, se_mean, sigma,
                        column_min, column_25, column_50, column_75, column_max, column_sum, column_kurtosis, column_skewness, mean_minus_three_sigmas, mean_plus_three_sigmas])
        
    # creating the finished table
    df_res = pd.DataFrame(data = df_data, columns = df_cols).sort_values(by='name')
    
    return df_res
	
# Пример вызова функции для вывода описательной статистики
# order_stats=dataframe_statistics(orders)
# display(order_stats)

In [9]:
data_analysis(marketing_events)

Первые 5 строк датасета:


,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


----------------------------------------
Полных дубликатов строк в датасете не обнаружено.
----------------------------------------


In [10]:
data_analysis(new_users)

Первые 5 строк датасета:


,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


----------------------------------------
Полных дубликатов строк в датасете не обнаружено.
----------------------------------------


In [11]:
data_analysis(events)

Первые 5 строк датасета:


,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


----------------------------------------
Полных дубликатов строк в датасете не обнаружено.
----------------------------------------


In [12]:
data_analysis(participants)

Первые 5 строк датасета:


,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


----------------------------------------
Полных дубликатов строк в датасете не обнаружено.
----------------------------------------


Посмотрим в выведенных таблицах на особенности распределения переменных 

In [13]:
dataframe_statistics(marketing_events)

,name,dtype,count,na_count,zero_count,unique_count,neg_count,mean,se_mean,sigma,min,25%,50%,75%,max,sum,kurtosis-3,adj_skewness,mean_-3sigma,mean_+3sigma
3,finish_dt,object,14,0,0,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,name,object,14,0,0,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,regions,object,14,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,start_dt,object,14,0,0,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


В таблице marketing_events мы наблюдаем календарь маркетинговых событий, представляющий собой датасет из 4 переменных: название события, регион, даты начала и окончания. Переменные с датами мы приведем к типу datetime. Всего 14 событий.

In [14]:
dataframe_statistics(new_users)

,name,dtype,count,na_count,zero_count,unique_count,neg_count,mean,se_mean,sigma,min,25%,50%,75%,max,sum,kurtosis-3,adj_skewness,mean_-3sigma,mean_+3sigma
3,device,object,61733,0,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,first_date,object,61733,0,0,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,region,object,61733,0,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,user_id,object,61733,0,0,61733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


В таблице new_users мы наблюдаем информацию по зарегистрировавшимся новым пользователям. Переменная device указывает на тип устройства с которого происходила регистрация, переменная region, вероятно, регион ip пользователя, переменная user_id - id пользователя, first_date - дата регистрации. Всего в датасете информация о 61733 пользователях. Переменную first_date придется привести к типу datetime.

In [15]:
dataframe_statistics(events)

,name,dtype,count,na_count,zero_count,unique_count,neg_count,mean,se_mean,sigma,min,25%,50%,75%,max,sum,kurtosis-3,adj_skewness,mean_-3sigma,mean_+3sigma
3,details,float64,62740,377577,0,4,0.0,23.88,0.29,72.18,4.99,4.99,4.99,9.99,499.99,1498082.6,33.836,5.658,-192.66,240.42
1,event_dt,object,440317,0,0,267268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,event_name,object,440317,0,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,user_id,object,440317,0,0,58703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


В таблице events содержится информация о пользовательских событиях. Всего в ней информация о 58703 пользователях и о 440317 событиях. Колонка event_name описывает тип события (login, product_page, product_card и purchase), колонка details содержит информацию о стоимости покупки в долларах для события purchase (со значениями, распределенными от 4.99 до 499.99). Пропуски в колонке details обусловлены отсутствием каких-либо деталей для остальных событий, у нас пока нет причин их трогать. Колонка event_dt содержит дату события и ее придется привести к типу datetime. 

In [16]:
dataframe_statistics(participants)

,name,dtype,count,na_count,zero_count,unique_count,neg_count,mean,se_mean,sigma,min,25%,50%,75%,max,sum,kurtosis-3,adj_skewness,mean_-3sigma,mean_+3sigma
2,ab_test,object,18268,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,group,object,18268,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,user_id,object,18268,0,0,16666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


В таблице participants содержится информация о 16666 пользователях, которые вошли в 2 по 2 группы тестов (переменная ab_test отвечает за вариант теста, group за группу теста) всего на 18268 человек. Это означает, что между какими-то группами будут пересечения, нам их потребуется установить.

## Предобработка и анализ переменных

Переведем переменные с датой в тип datetime

In [17]:
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])
new_users['first_date'] = pd.to_datetime(new_users['first_date'])
events['event_dt'] = pd.to_datetime(events['event_dt'])
events['event_dt']=events['event_dt'].dt.date
events['event_dt'] = pd.to_datetime(events['event_dt'])

### Участие пользователей в других тестах

Оценим характер дубликатов по id пользователя

In [18]:
# Найдем дубликаты по user_id
duplicates = participants[participants.duplicated(subset='user_id', keep=False)]

# Сортируем дубликаты для вывода парами
sorted_duplicates = duplicates.sort_values('user_id')

# Выводим дубликаты парами
for i in range(0, len(sorted_duplicates), 2):
    print(sorted_duplicates.iloc[i:i+2])

                user_id group                  ab_test
17892  001064FEAAB631A1     B        interface_eu_test
235    001064FEAAB631A1     B  recommender_system_test
                user_id group                  ab_test
16961  00341D8401F0F665     A        interface_eu_test
2137   00341D8401F0F665     A  recommender_system_test
               user_id group                  ab_test
8143  003B6786B4FF5B03     A        interface_eu_test
3156  003B6786B4FF5B03     A  recommender_system_test
                user_id group                  ab_test
4768   0082295A41A867B5     A  recommender_system_test
14161  0082295A41A867B5     B        interface_eu_test
                user_id group                  ab_test
15562  00E68F103C66C1F7     B        interface_eu_test
4074   00E68F103C66C1F7     A  recommender_system_test
                user_id group                  ab_test
13517  00EFA157F7B6E1C4     A        interface_eu_test
2541   00EFA157F7B6E1C4     A  recommender_system_test
             

                user_id group                  ab_test
17617  0B0D84A866817D84     B        interface_eu_test
5487   0B0D84A866817D84     A  recommender_system_test
                user_id group                  ab_test
12465  0B1A00B4B0AAA92A     B        interface_eu_test
2704   0B1A00B4B0AAA92A     A  recommender_system_test
                user_id group                  ab_test
5942   0B3D0ECA098E1B8F     B  recommender_system_test
12084  0B3D0ECA098E1B8F     B        interface_eu_test
               user_id group                  ab_test
8509  0B6A69ADBF740022     A        interface_eu_test
4966  0B6A69ADBF740022     B  recommender_system_test
                user_id group                  ab_test
14046  0BA00E790AA510C1     B        interface_eu_test
5882   0BA00E790AA510C1     A  recommender_system_test
                user_id group                  ab_test
16851  0BC7C730D40D19D3     B        interface_eu_test
801    0BC7C730D40D19D3     A  recommender_system_test
             

               user_id group                  ab_test
9469  14D84579BCBAD800     A        interface_eu_test
6221  14D84579BCBAD800     A  recommender_system_test
                user_id group                  ab_test
10461  14E7C76B7AB8822A     A        interface_eu_test
2601   14E7C76B7AB8822A     A  recommender_system_test
                user_id group                  ab_test
5999   14EA5C10D98DCC49     B  recommender_system_test
11454  14EA5C10D98DCC49     B        interface_eu_test
                user_id group                  ab_test
11378  15BB5FA78262CFCE     B        interface_eu_test
726    15BB5FA78262CFCE     B  recommender_system_test
               user_id group                  ab_test
1751  1632E8454F655D06     B  recommender_system_test
7025  1632E8454F655D06     A        interface_eu_test
                user_id group                  ab_test
1681   1675B921AC721DBE     A  recommender_system_test
11729  1675B921AC721DBE     B        interface_eu_test
                

                user_id group                  ab_test
6317   28F165B39D160BC5     A  recommender_system_test
13760  28F165B39D160BC5     A        interface_eu_test
               user_id group                  ab_test
2187  290AA8D9707F41D8     B  recommender_system_test
7284  290AA8D9707F41D8     B        interface_eu_test
               user_id group                  ab_test
2865  2926BF724D5C53EF     A  recommender_system_test
7975  2926BF724D5C53EF     B        interface_eu_test
                user_id group                  ab_test
15133  29408D41AEA456C2     A        interface_eu_test
2249   29408D41AEA456C2     A  recommender_system_test
                user_id group                  ab_test
15611  29535733A8B8C27A     A        interface_eu_test
1619   29535733A8B8C27A     B  recommender_system_test
                user_id group                  ab_test
2397   295739D983D3C350     B  recommender_system_test
17043  295739D983D3C350     A        interface_eu_test
                

               user_id group                  ab_test
1289  32ADE395AD180A46     B  recommender_system_test
6783  32ADE395AD180A46     A        interface_eu_test
                user_id group                  ab_test
13921  32AF1446FF633378     B        interface_eu_test
1391   32AF1446FF633378     A  recommender_system_test
                user_id group                  ab_test
14295  32E13C1C0F6AC96E     B        interface_eu_test
4709   32E13C1C0F6AC96E     A  recommender_system_test
               user_id group                  ab_test
5283  3340B98CBEAEE766     A  recommender_system_test
9552  3340B98CBEAEE766     B        interface_eu_test
                user_id group                  ab_test
12017  3341349F0FD06C14     A        interface_eu_test
4399   3341349F0FD06C14     A  recommender_system_test
                user_id group                  ab_test
1995   3382B624CF985A13     A  recommender_system_test
14099  3382B624CF985A13     A        interface_eu_test
                

...

                user_id group                  ab_test
4823   88FB1EDF3E249223     B  recommender_system_test
17464  88FB1EDF3E249223     A        interface_eu_test
                user_id group                  ab_test
2861   891458DEF142C0A3     A  recommender_system_test
10688  891458DEF142C0A3     A        interface_eu_test
                user_id group                  ab_test
1290   892061952E6224DF     B  recommender_system_test
10993  892061952E6224DF     B        interface_eu_test
               user_id group                  ab_test
3196  894A2506B3A50D03     B  recommender_system_test
7744  894A2506B3A50D03     A        interface_eu_test
               user_id group                  ab_test
4165  89DBC4C20AAE2450     B  recommender_system_test
9912  89DBC4C20AAE2450     A        interface_eu_test
                user_id group                  ab_test
12475  8A0D2C0C98226F28     B        interface_eu_test
2433   8A0D2C0C98226F28     B  recommender_system_test
               u

               user_id group                  ab_test
3085  9A8CD4B32D7511F1     A  recommender_system_test
9342  9A8CD4B32D7511F1     B        interface_eu_test
                user_id group                  ab_test
17529  9AE73C8017160225     A        interface_eu_test
5986   9AE73C8017160225     B  recommender_system_test
                user_id group                  ab_test
14549  9AFA49F655017427     B        interface_eu_test
4946   9AFA49F655017427     B  recommender_system_test
               user_id group                  ab_test
8910  9B802A12D586586F     A        interface_eu_test
2022  9B802A12D586586F     B  recommender_system_test
                user_id group                  ab_test
698    9BABE80DC9F2BE91     A  recommender_system_test
10513  9BABE80DC9F2BE91     B        interface_eu_test
               user_id group                  ab_test
9103  9BD9E79856D54059     B        interface_eu_test
6065  9BD9E79856D54059     B  recommender_system_test
                use

                user_id group                  ab_test
11944  B37F427F93BCA1E3     A        interface_eu_test
3729   B37F427F93BCA1E3     A  recommender_system_test
               user_id group                  ab_test
9034  B3A2485649E4A012     A        interface_eu_test
10    B3A2485649E4A012     A  recommender_system_test
                user_id group                  ab_test
15211  B3AEC82CC231D137     A        interface_eu_test
5456   B3AEC82CC231D137     A  recommender_system_test
               user_id group                  ab_test
5508  B3AF8DD078D5F58F     B  recommender_system_test
8391  B3AF8DD078D5F58F     B        interface_eu_test
                user_id group                  ab_test
79     B3C1FF8D21EAC16B     B  recommender_system_test
17984  B3C1FF8D21EAC16B     B        interface_eu_test
               user_id group                  ab_test
8719  B3D8D3BB3ED8F850     A        interface_eu_test
273   B3D8D3BB3ED8F850     B  recommender_system_test
                use

                user_id group                  ab_test
10088  CF849D6380A28CA9     A        interface_eu_test
5842   CF849D6380A28CA9     A  recommender_system_test
                user_id group                  ab_test
15033  CFA945B0FCFE90D9     B        interface_eu_test
3133   CFA945B0FCFE90D9     B  recommender_system_test
               user_id group                  ab_test
165   CFDED9167B27A57F     A  recommender_system_test
9722  CFDED9167B27A57F     B        interface_eu_test
               user_id group                  ab_test
8816  D043314C33C9ADBD     A        interface_eu_test
3823  D043314C33C9ADBD     A  recommender_system_test
                user_id group                  ab_test
11227  D0533ECF2A67B35C     A        interface_eu_test
3484   D0533ECF2A67B35C     A  recommender_system_test
               user_id group                  ab_test
2057  D0562837FBB58AC5     A  recommender_system_test
6858  D0562837FBB58AC5     B        interface_eu_test
                use

                user_id group                  ab_test
16475  E6FE73B8B903B31B     B        interface_eu_test
2800   E6FE73B8B903B31B     A  recommender_system_test
               user_id group                  ab_test
9660  E6FF12D918041A75     A        interface_eu_test
4732  E6FF12D918041A75     B  recommender_system_test
               user_id group                  ab_test
8148  E722C2D81148A653     B        interface_eu_test
3846  E722C2D81148A653     B  recommender_system_test
                user_id group                  ab_test
11013  E77F99BCF9D8AB96     B        interface_eu_test
5887   E77F99BCF9D8AB96     B  recommender_system_test
                user_id group                  ab_test
842    E78902CB3CAA29F7     B  recommender_system_test
10704  E78902CB3CAA29F7     A        interface_eu_test
                user_id group                  ab_test
4387   E7CC63BF20618321     B  recommender_system_test
14996  E7CC63BF20618321     A        interface_eu_test
                

In [19]:
# Получим размер исходной таблицы
total_rows_original = participants.shape[0]

# Удалим дубликаты пользователей
participants = participants.drop_duplicates(subset='user_id', keep='first')

# Получим размер таблицы после удаления дубликатов
total_rows_unique = participants.shape[0]

# Вычислим количество удаленных дубликатов
duplicates_removed = total_rows_original - total_rows_unique

# Выведем количество удаленных дубликатов
print(f'Количество удаленных дубликатов: {duplicates_removed}')

Количество удаленных дубликатов: 1602


При подготовке к проведению A/B теста необходимо убедиться в строгом контроле уровней независимой переменной на выборках. Это подразумевает то, что пользователи должны быть строго поделены по методу экспериментального воздействия. Говоря проще, они должны принадлежать либо к группе А, либо к группе В. Если они внезапно оказались в обеих, это испорченные испытуемые с эффектом тестирования, они снижают внешнюю валидность исследования и от них следует избавляться. С другой стороны, у нас пользователи набирались одновременно для двух различных тестов. Как мы видим по выводу пар дубликатов, пользователи пересекались между различыми группами двух тестов, что дает малопрогнозируемые результаты и от таких пользователей приходится избавляться по тем же причинам: это испорченные испытуемые.

### Временные интервалы

Оценим временной интервал по датам событий в датасете events

In [20]:
start_date_events = events['event_dt'].min()
end_date_events = events['event_dt'].max()

print("Дата начала представленных данных:", start_date_events)
print("Дата окончания представленных данных:", end_date_events)


Дата начала представленных данных: 2020-12-07 00:00:00
Дата окончания представленных данных: 2020-12-30 00:00:00


События в датасете происходили с 7 по 30 декабря. ТЗ предполагало что они продлятся до 4 января, но не срослось.

Оценим временной интервал по датам регистрации в датасете new_users

In [21]:
start_date_new_users = new_users['first_date'].min()
end_date_new_users = new_users['first_date'].max()

print("Дата начала регистрации участников:", start_date_new_users)
print("Дата окончания регистрации участников:", end_date_new_users)


Дата начала регистрации участников: 2020-12-07 00:00:00
Дата окончания регистрации участников: 2020-12-23 00:00:00


Пользователи регистрировались с 7 декабря по 23 декабря. ТЗ предполагало что 21 декабря будет последним днем регистрации (чтобы до 4 января у пользователей было время совершить хоть какие-то события в течение 14 дней), но раз с 4м января не задалось, то, как говорится, "горит сарай - гори и хата"... 

Взглянем в глаза пользователям, не уложившимся в заданные сроки регистрации

In [22]:
is_within_period = (new_users['first_date'] >= '2020-12-07') & (new_users['first_date'] <= '2020-12-21')
# Выберем только пользователей, не соответствующих периоду
non_compliant_users = new_users[~is_within_period]

# Выведем таблицу с этими пользователями
print(non_compliant_users.head())

                user_id first_date     region   device
23821  5815F7ECE74D949F 2020-12-22        CIS       PC
23822  32EAEA5E903E3BC1 2020-12-22  N.America  Android
23823  9DF7A3C46487EF0B 2020-12-22         EU  Android
23824  ADE98C6440423287 2020-12-22         EU   iPhone
23825  5A5833D3AEA75255 2020-12-22  N.America       PC


Попрощаемся с ними

In [23]:
# Удалим пользователей, не соответствующих периоду, из таблицы с участниками теста
participants_cleaned = participants[participants['user_id'].isin(new_users[is_within_period]['user_id'])]

# Посчитаем количество удаленных пользователей
deleted_users_count = len(participants) - len(participants_cleaned)

# Выведем количество удаленных пользователей
print(f'Количество удаленных пользователей из таблицы участников теста: {deleted_users_count}')

Количество удаленных пользователей из таблицы участников теста: 1002


Оставим в датасете лишь тех пользователей что участвовали в нашем исследовании

In [24]:
# Оставим только участников теста "recommender_system_test" в participants_cleaned
participants_cleaned = participants_cleaned[participants_cleaned['ab_test'] == 'recommender_system_test']

### Регион регистрации

Оценим пользователей по принадлежности к целевому региону

In [25]:
# Выделим пользователей из целевого региона (EU)
target_region_users = new_users[new_users['region'] == 'EU']

# Проверим, что все пользователи из participants_cleaned представляют целевой регион
all_in_target_region = participants_cleaned['user_id'].isin(target_region_users['user_id']).all()

# Выведем результат проверки
if all_in_target_region:
    print("Все пользователи, участвующие в тесте, представляют целевой регион (EU).")
else:
    print("Не все пользователи, участвующие в тесте, представляют целевой регион (EU).")

Не все пользователи, участвующие в тесте, представляют целевой регион (EU).


In [26]:
# Выберем пользователей, не представляющих целевой регион (EU)
non_target_region_users = participants_cleaned[~participants_cleaned['user_id'].isin(target_region_users['user_id'])]

# Получим регионы этих пользователей из таблицы new_users
non_target_region_user_regions = new_users[new_users['user_id'].isin(non_target_region_users['user_id'])]

# Выведем таблицу с этими пользователями и их регионами
print(non_target_region_user_regions[['user_id', 'region']].head())

              user_id     region
40   29C92313A98B1176       APAC
45   7D1BFB181017EB46        CIS
235  C6EB47582A64E176        CIS
261  736990A3BC742A65        CIS
532  800AF45A68291849  N.America


Избавимся от пользователей из нецелевых регионов

In [27]:
# Подсчитаем количество пользователей, которые будут удалены
count_deleted_users = len(non_target_region_users)

# Выведем количество пользователей, которые будут удалены
print(f'Количество пользователей, которые будут удалены: {count_deleted_users}')

# Удалим пользователей, не представляющих целевой регион, из participants_cleaned
participants_cleaned = participants_cleaned[participants_cleaned['user_id'].isin(target_region_users['user_id'])]

# Подсчитаем количество пользователей, которые остались после удаления
count_remaining_users = len(participants_cleaned)

# Выведем количество удаленных пользователей и количество оставшихся пользователей
print(f'Количество удаленных пользователей: {count_deleted_users}')
print(f'Количество оставшихся пользователей: {count_remaining_users}')


Количество пользователей, которые будут удалены: 350
Количество удаленных пользователей: 350
Количество оставшихся пользователей: 6351


Оценим процент участником теста от общего числа новых пользователей из целевого региона

In [28]:
# Определим период набора пользователей в тест
start_date = '2020-12-07'
end_date = '2020-12-21'

# Выберем только пользователей из целевого региона (EU) и зарегистрированных в правильный период
target_region_users_in_period = target_region_users[
    (target_region_users['first_date'] >= start_date) &
    (target_region_users['first_date'] <= end_date)
]

# Подсчитаем общее количество пользователей из целевого региона, зарегистрированных в период набора пользователей в тест
total_target_region_users_in_period = len(target_region_users_in_period)

# Подсчитаем общее количество участников теста
total_test_participants = len(participants_cleaned)

# Рассчитаем процент участников теста от общего числа пользователей из целевого региона в правильный период
percentage_test_participants = (total_test_participants / total_target_region_users_in_period) * 100

# Выведем результат
print(f'Процент участников теста от общего числа пользователей из целевого региона: {percentage_test_participants:.2f}%')


Процент участников теста от общего числа пользователей из целевого региона: 15.00%


In [29]:
dataframe_statistics(participants_cleaned)

,name,dtype,count,na_count,zero_count,unique_count,neg_count,mean,se_mean,sigma,min,25%,50%,75%,max,sum,kurtosis-3,adj_skewness,mean_-3sigma,mean_+3sigma
2,ab_test,object,6351,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,group,object,6351,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,user_id,object,6351,0,0,6351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Динамика набора пользователей

In [30]:
# Вывести количество строк в исходных таблицах
print("Количество строк в participants_cleaned:", len(participants_cleaned))
print("Количество строк в new_users:", len(new_users))

# Объединить таблицы participants_cleaned и new_users по столбцу 'user_id' с использованием left join
participants_cleaned = participants_cleaned.merge(new_users, on='user_id', how='left')

# Вывести количество строк в объединенной таблице
print("Количество строк в объединенной таблице:", len(participants_cleaned))

# Выведем первые несколько строк обновленной таблицы для проверки
print(participants_cleaned.head())


Количество строк в participants_cleaned: 6351
Количество строк в new_users: 61733
Количество строк в объединенной таблице: 6351
            user_id group                  ab_test first_date region  device
0  D1ABA3E2887B6A73     A  recommender_system_test 2020-12-07     EU      PC
1  A7A3664BD6242119     A  recommender_system_test 2020-12-20     EU  iPhone
2  DABC14FDDFADD29E     A  recommender_system_test 2020-12-08     EU     Mac
3  04988C5DF189632E     A  recommender_system_test 2020-12-14     EU  iPhone
4  482F14783456D21B     B  recommender_system_test 2020-12-14     EU      PC


In [31]:
dataframe_statistics(participants_cleaned)

,name,dtype,count,na_count,zero_count,unique_count,neg_count,mean,se_mean,sigma,min,25%,50%,75%,max,sum,kurtosis-3,adj_skewness,mean_-3sigma,mean_+3sigma
2,ab_test,object,6351,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,device,object,6351,0,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,first_date,datetime64[ns],6351,0,0,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,group,object,6351,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,region,object,6351,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,user_id,object,6351,0,0,6351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Оценим динамику регистрации в разрезе 2х групп

In [33]:
# Преобразуем столбец 'first_date' в строковый формат
participants_cleaned['first_date'] = participants_cleaned['first_date'].dt.strftime('%Y-%m-%d')

# Группируем участников теста по дате регистрации и группе
participants_grouped = participants_cleaned.groupby(['first_date', 'group']).count().reset_index()

# Создаем интерактивный линейный график
fig = px.line(participants_grouped, x='first_date', y='user_id', color='group',
              labels={'first_date': 'Дата регистрации', 'user_id': 'Количество пользователей'},
              title='Динамика набора пользователей в группы теста')

# Настраиваем внешний вид графика
fig.update_xaxes(type='category')
fig.update_xaxes(tickangle=45)
fig.update_layout(showlegend=True, legend_title_text='Группа')

# Отображаем график
fig.show()


In [34]:
participants_cleaned['first_date'] = pd.to_datetime(participants_cleaned['first_date'])

In [35]:
# Подсчитаем количество пользователей в каждой группе
group_counts = participants_cleaned['group'].value_counts()

# Подсчитаем процентное соотношение пользователей в каждой группе
group_percentages = (group_counts / group_counts.sum()) * 100

# Выведем результат
print("Количество пользователей в каждой группе:")
print(group_counts)
print("\nПроцентное соотношение пользователей в каждой группе:")
print(group_percentages)

Количество пользователей в каждой группе:
group
A    3634
B    2717
Name: count, dtype: int64

Процентное соотношение пользователей в каждой группе:
group
A    57.219336
B    42.780664
Name: count, dtype: float64


Разбиение на группы происходило равномерно в течение 14 дней регистрации, но в группу Б стабильно попадало меньше пользователей.

In [36]:
# Общее количество пользователей
total_users = len(participants_cleaned)

# Количество пользователей в каждой группе
group_counts = participants_cleaned['group'].value_counts().reset_index()
group_counts.columns = ['group', 'count']

In [37]:
count = np.array([group_counts[group_counts['group'] == 'A']['count'].values[0], group_counts[group_counts['group'] == 'B']['count'].values[0]])
nobs = np.array([total_users, total_users])

# Выполнение двухпропорционального Z-теста
z_stat, p_value = proportions_ztest(count, nobs)

# Вывод результатов
print(f'Z-statistic = {z_stat:.3f}; p = {p_value:.3f}')

Z-statistic = 16.273; p = 0.000


Однако если не брать в расчет разрез времени, общее распределение по группам равномерным назвать трудно.

Посмотрим как разбиваются устройства с которых происходила регистрация по группам

In [38]:
# Группируем пользователей по устройствам и группам теста и считаем количество пользователей
device_group_counts = participants_cleaned.groupby(['device', 'group']).size().reset_index(name='count')

# Создаем сводную таблицу для удобства анализа
pivot_table = pd.pivot_table(device_group_counts, values='count', index='device', columns='group', fill_value=0)

# Нормируем количество пользователей по устройствам в каждой группе
pivot_table['A_percent'] = (pivot_table['A'] / pivot_table['A'].sum()) * 100
pivot_table['B_percent'] = (pivot_table['B'] / pivot_table['B'].sum()) * 100

# Выводим сводную таблицу
print(pivot_table)

group         A       B  A_percent  B_percent
device                                       
Android  1590.0  1228.0  43.753440  45.196908
Mac       354.0   250.0   9.741332   9.201325
PC        964.0   657.0  26.527243  24.181082
iPhone    726.0   582.0  19.977986  21.420685


In [39]:
# Нормируем значения по размеру группы
normalized_data = participants_cleaned.groupby('group')['device'].value_counts(normalize=True).reset_index(name='percentage')

# Создаем диаграмму
fig = px.bar(normalized_data, x='device', y='percentage', color='group',
             title='Распределение пользователей по устройствам с разбиением по группе',
             labels={'device': 'Устройство', 'percentage': 'Процент', 'group': 'Группа'})

# Отобразим график
fig.show()


Распределение пользователей по устройствам регистрации сносное.

### Исследование пользовательской активности

In [40]:
# Вывести количество строк в исходных таблицах
print("Количество строк в participants_cleaned:", len(participants_cleaned))
print("Количество строк в events:", len(events))

# Объединить таблицы participants_cleaned и events по столбцу 'user_id' с использованием left join
participants_cleaned = participants_cleaned.merge(events, on='user_id', how='left')

# Вывести количество строк в объединенной таблице
print("Количество строк в объединенной таблице:", len(participants_cleaned))

# Выведем первые несколько строк объединенной таблицы для проверки
display(participants_cleaned.head())


Количество строк в participants_cleaned: 6351
Количество строк в events: 440317
Количество строк в объединенной таблице: 26290


,user_id,group,ab_test,first_date,region,device,event_dt,event_name,details
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07,purchase,99.99
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-25,purchase,4.99
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07,product_cart,NaN
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-25,product_cart,NaN
4,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07,product_page,NaN


Рассмотрим пользовательские воронки в наших датасетах

In [41]:
unique_events = events['event_name'].unique()
print(unique_events)

['purchase' 'product_cart' 'product_page' 'login']


In [42]:
total_unique_users = events['user_id'].nunique()
print("Общее число уникальных пользователей:", total_unique_users)

Общее число уникальных пользователей: 58703


In [43]:
# Создаем данные для воронки
event_names = ['login', 'product_page', 'product_cart', 'purchase']
user_counts = [events[events['event_name'] == event_name]['user_id'].nunique() for event_name in event_names]

# Создаем график пользовательской воронки
fig_user = go.Figure(go.Funnel(
    y=event_names,
    x=user_counts,
    textinfo="value+percent initial",
    name="Продуктовая воронка"
))
fig_user.update_layout(title="Продуктовая воронка")

# Отображаем график пользовательской воронки
fig_user.show()


In [44]:
unique_user_counts = events.groupby('event_name')['user_id'].nunique()

for event_name, count in unique_user_counts.items():
    print(f'Уникальных пользователей для события "{event_name}": {count}')


Уникальных пользователей для события "login": 58697
Уникальных пользователей для события "product_cart": 19284
Уникальных пользователей для события "product_page": 38929
Уникальных пользователей для события "purchase": 19569


Как мы видим уже по внешнему виду, воронка не является строгой, так как покупки совершило больше чем зашло в корзину.

Посмотрим на воронки для тестовых групп

In [45]:
# Создаем список групп
groups = participants_cleaned['group'].unique()

# Создаем списки пользователей для каждой группы
user_lists = []

for group in groups:
    user_list = participants_cleaned[participants_cleaned['group'] == group]['user_id'].unique()
    user_lists.append(user_list)

# События, для которых вы хотите создать воронку
event_names = ['login', 'product_page', 'product_cart', 'purchase']

# Создаем воронки для каждой группы
group_funnel_figures = []

for i, group in enumerate(groups):
    user_counts = []
    for event_name in event_names:
        count = events[(events['user_id'].isin(user_lists[i])) & (events['event_name'] == event_name)]['user_id'].nunique()
        user_counts.append(count)
    
    fig = go.Figure(go.Funnel(
        y=event_names,
        x=user_counts,
        textinfo="value+percent initial",
        name=f"Воронка группы {group}"
    ))
    fig.update_layout(title=f"Воронка группы {group}")
    group_funnel_figures.append(fig)

# Отображаем графики пользовательских воронок для каждой группы
for fig in group_funnel_figures:
    fig.show()


Караул, пропали зарегистрированные пользователи, никто не видел? Вроде их всего 6 с лишним тысяч только что было...

In [46]:
# список уникальных пользователей, зарегистрированных в каждой группе
users_registered_group_A = participants_cleaned[participants_cleaned['group'] == 'A']['user_id'].unique()
users_registered_group_B = participants_cleaned[participants_cleaned['group'] == 'B']['user_id'].unique()

# список уникальных пользователей, совершивших события
users_with_events = events['user_id'].unique()

# Найдем пользователей, зарегистрированных, но не совершивших события
users_without_events_group_A = len(set(users_registered_group_A) - set(users_with_events))
users_without_events_group_B = len(set(users_registered_group_B) - set(users_with_events))

# Вычислим процент пользователей без событий относительно зарегистрированных в группе
total_users_group_A = len(users_registered_group_A)
total_users_group_B = len(users_registered_group_B)

percent_without_events_group_A = (users_without_events_group_A / total_users_group_A) * 100
percent_without_events_group_B = (users_without_events_group_B / total_users_group_B) * 100

# Выведем количество и процент пользователей без событий после регистрации
print("Количество пользователей без событий после регистрации в группе A:", users_without_events_group_A)
print("Процент пользователей без событий после регистрации в группе A:", percent_without_events_group_A, "%")

print("Количество пользователей без событий после регистрации в группе B:", users_without_events_group_B)
print("Процент пользователей без событий после регистрации в группе B:", percent_without_events_group_B, "%")

Количество пользователей без событий после регистрации в группе A: 1030
Процент пользователей без событий после регистрации в группе A: 28.343423225096316 %
Количество пользователей без событий после регистрации в группе B: 1840
Процент пользователей без событий после регистрации в группе B: 67.72175193227825 %


В группе Б как-то не очень просто, по-видимому, совершить событие. Им и при наборе не особо перепало, и после регистрации снова продолжилось...

Исследуем пользователей, не совершавших события

In [47]:
# Получим уникальных пользователей без событий для группы A
users_without_events_group_A = set(users_registered_group_A) - set(users_with_events)

# Получим уникальных пользователей без событий для группы B
users_without_events_group_B = set(users_registered_group_B) - set(users_with_events)

# Создадим подмножество participants_cleaned только для уникальных пользователей без событий
participants_without_events_group_A = participants_cleaned[(participants_cleaned['group'] == 'A') & (participants_cleaned['user_id'].isin(users_without_events_group_A))]
participants_without_events_group_B = participants_cleaned[(participants_cleaned['group'] == 'B') & (participants_cleaned['user_id'].isin(users_without_events_group_B))]

# Получим распределение типов устройств для уникальных пользователей без событий
device_distribution_group_A = participants_without_events_group_A['device'].value_counts()
device_distribution_group_B = participants_without_events_group_B['device'].value_counts()

print("Распределение типов устройств среди уникальных пользователей без событий в группе A:")
print(device_distribution_group_A)

print("Распределение типов устройств среди уникальных пользователей без событий в группе B:")
print(device_distribution_group_B)



Распределение типов устройств среди уникальных пользователей без событий в группе A:
device
Android    451
PC         275
iPhone     205
Mac         99
Name: count, dtype: int64
Распределение типов устройств среди уникальных пользователей без событий в группе B:
device
Android    823
PC         445
iPhone     396
Mac        176
Name: count, dtype: int64


Распределение по устройствам приемлемое

Как мы видим, доля пользователей, не совершивших ни одного события после регистрации в группе Б значительно выше чем доля таких же пользователей в группе А. С типом устройства это, очевидно, никак не связано.

In [48]:
# Создадим подмножество данных для уникальных пользователей без событий в обеих группах
filtered_participants_A = participants_cleaned[(participants_cleaned['group'] == 'A') & (participants_cleaned['user_id'].isin(users_without_events_group_A))]
filtered_participants_B = participants_cleaned[(participants_cleaned['group'] == 'B') & (participants_cleaned['user_id'].isin(users_without_events_group_B))]

# Объединим данные для обеих групп
combined_data = pd.concat([filtered_participants_A, filtered_participants_B])

# Создадим гистограмму 
fig = px.histogram(combined_data, x='first_date', color='group', title="Распределение дат регистрации уникальных пользователей без событий",
                   labels={'first_date': 'Дата регистрации', 'count': 'Количество пользователей'})

fig.show()



C:\Users\Sergey\anaconda3\envs\da_practicum_env\lib\site-packages\_plotly_utils\basevalidators.py:107: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Убедимся что с момента изучения динамики регистрации мы не потеряли часть данных

In [49]:
# Найдем минимальную и максимальную дату регистрации в группе A
min_date_group_A = participants_cleaned[participants_cleaned['group'] == 'A']['first_date'].min()
max_date_group_A = participants_cleaned[participants_cleaned['group'] == 'A']['first_date'].max()

# Найдем минимальную и максимальную дату регистрации в группе B
min_date_group_B = participants_cleaned[participants_cleaned['group'] == 'B']['first_date'].min()
max_date_group_B = participants_cleaned[participants_cleaned['group'] == 'B']['first_date'].max()

print("Диапазон дат регистрации в группе A: от", min_date_group_A, "до", max_date_group_A)
print("Диапазон дат регистрации в группе B: от", min_date_group_B, "до", max_date_group_B)


Диапазон дат регистрации в группе A: от 2020-12-07 00:00:00 до 2020-12-21 00:00:00
Диапазон дат регистрации в группе B: от 2020-12-07 00:00:00 до 2020-12-21 00:00:00


Нет, не потеряли

Выведем отдельный датасет с пользователями из тестовых групп, не совершавшими никаких событий

In [50]:
users_without_events_df = participants_cleaned[participants_cleaned['event_dt'].isna()]

Отсечем всех пользователей, зарегистрированных после 13 декабря

In [51]:
# Установим дату 14 декабря 2020 года
cutoff_date = pd.to_datetime('2020-12-14')

# Обрежем датасет до 14 декабря 2020 года
trimmed_users_without_events_df = users_without_events_df[users_without_events_df['first_date'] < cutoff_date]

# Подсчитаем количество пользователей по группам
user_count_by_group = trimmed_users_without_events_df.groupby('group')['user_id'].count()

# Выведем количество пользователей в каждой группе
print("Количество пользователей в группе A, зарегистрированных до 13 декабря включительно, не совершивших ни одного события:", user_count_by_group['A'])
print("Количество пользователей в группе B, зарегистрированных до 13 декабря включительно, не совершивших ни одного события:", user_count_by_group['B'])


Количество пользователей в группе A, зарегистрированных до 13 декабря включительно, не совершивших ни одного события: 1030
Количество пользователей в группе B, зарегистрированных до 13 декабря включительно, не совершивших ни одного события: 818


Как легко заметить, в группе А среди зарегистрированных после 13 декабря, тех кто не совершил бы ни одного события нет. В то же самое время, в группе Б такие пользователи распределены по всему периоду регистрации. В данной связи, основная дилемма в том что критичнее: то что в группе А все 100% пользователей, зарегистрировавшихся после 13 декабря совершили какое-то действие, или то что существует такая разница в динамике между группами. Как так случилось понять в отрыве от процедуры тестирования невозможно. Сделаем допущение о том, что 100% выполнение действий после регистрации это норма для данного сервиса. Тогда поведение для пользователей группы А, зарегистрированных после 13 декабря, нормально, а поведение пользователей группы А, зарегистрированных до 14 декабря и пользователей группы Б является отклонением. В этом случае, можно предположить наличие технических проблем, устраненных только для группы А таким образом, что они не затронули группу зарегистрированных с 14 декабря. В то же время для группы Б эти проблемы так и не были устранены. Например могла сбоить сама процедура аутентификации клиента, что приводило к тому, что часть успешно зарегистрированных клиентов не могла залогиниться. В этом случае, пользователей зарегистрированных 14 декабря и позднее следует удалить из теста: мы не знаем доподлинно в чем состоит модная улучшенная рекомендательная система, но вряд ли именно в том чтобы не давать пользователям покупать товары, а следовательно, внутренняя валидность эксперимента не была проконтролирована и появилась побочная переменная в виде технических сложностей лишь у экспериментальной группы. Т.о сравнивать результаты тех кто регистрировался с 7 по 13 можно, позднее нет. 

In [52]:
# Установим дату 14 декабря 2020 года
cutoff_date = pd.to_datetime('2020-12-14')

# Обрежем датасет participants_cleaned, чтобы удалить данные, начиная с 14 декабря 2020 года
trimmed_participants_cleaned = participants_cleaned[participants_cleaned['first_date'] < cutoff_date]

In [53]:
# Удалите дубликаты по столбцу "user_id" для получения уникальных пользователей
unique_participants = participants_cleaned.drop_duplicates(subset='user_id')

# Группируем данные по группе и дате регистрации для уникальных пользователей
grouped_data = unique_participants.groupby(['group', 'first_date'])['user_id'].count().reset_index()

# Перебираем группы и выводим даты регистрации для каждой группы
for group_name, group_data in grouped_data.groupby('group'):
    print(f'Группа {group_name}:')
    print(group_data[['first_date', 'user_id']])
    print('\n')

Группа A:
   first_date  user_id
0  2020-12-07      349
1  2020-12-08      186
2  2020-12-09      143
3  2020-12-10      189
4  2020-12-11      157
5  2020-12-12      268
6  2020-12-13      323
7  2020-12-14      366
8  2020-12-15      202
9  2020-12-16      160
10 2020-12-17      202
11 2020-12-18      210
12 2020-12-19      226
13 2020-12-20      255
14 2020-12-21      398


Группа B:
   first_date  user_id
15 2020-12-07      259
16 2020-12-08      149
17 2020-12-09      105
18 2020-12-10      181
19 2020-12-11      106
20 2020-12-12      198
21 2020-12-13      228
22 2020-12-14      262
23 2020-12-15      141
24 2020-12-16      114
25 2020-12-17      138
26 2020-12-18      172
27 2020-12-19      173
28 2020-12-20      206
29 2020-12-21      285




In [54]:
# Подсчитаем число уникальных пользователей по группам
unique_users_group_A = trimmed_participants_cleaned[trimmed_participants_cleaned['group'] == 'A']['user_id'].nunique()
unique_users_group_B = trimmed_participants_cleaned[trimmed_participants_cleaned['group'] == 'B']['user_id'].nunique()

# Выведем число уникальных пользователей по группам
print("Число уникальных пользователей в группе A:", unique_users_group_A)
print("Число уникальных пользователей в группе B:", unique_users_group_B)


Число уникальных пользователей в группе A: 1615
Число уникальных пользователей в группе B: 1226


В уменьшенном датасете распределение по группам продолжает страдать...

In [55]:
count = np.array([user_count_by_group['A'], user_count_by_group['B']])
nobs = np.array([unique_users_group_A, unique_users_group_B])

# Выполнение двухпропорционального Z-теста
z_stat, p_value = proportions_ztest(count, nobs)

# Вывод результатов
print(f'Z-statistic = {z_stat:.3f}; p = {p_value:.3f}')

Z-statistic = -1.630; p = 0.103


Значимых различий по доле пользователей без событий мы не наблюдаем, так что нам они особо не помешают.

С одной стороны, пользователи без событий это показатель пользовательской конверсии: берешь и используешь. С другой стороны, результаты теста сводятся к продуктовой воронке, а пользователи без событий по очевидным причинам в нее не попадают. Иными словами, с одной стороны, эти пользователи нам никак не докучают, а с другой - позволяют получить дополнительное представление о привлекательности новой платежной воронки. Однако в рамках нашей задачи, очевидно, что эти пользователи испорчены, поскольку что-то им мешало. Нам мало что мешает их оставить в датасете, но, надо понимать, что не зная в точности как эта когорта пользователей была создана, мы не сможем сделать о них никаких выводов. Так или иначе, можно предположить, что регистрация у всех происходила более или менее одинакова, а события на выборке мы будем анализировать уже среди тех кто эти события совершал.

In [56]:
# Создаем копию датасета participants_cleaned с индексом non_trimmed
participants_cleaned_non_trimmed = participants_cleaned.copy()


In [57]:
participants_cleaned = trimmed_participants_cleaned

In [58]:
dataframe_statistics(participants_cleaned)

,name,dtype,count,na_count,zero_count,unique_count,neg_count,mean,se_mean,sigma,min,25%,50%,75%,max,sum,kurtosis-3,adj_skewness,mean_-3sigma,mean_+3sigma
2,ab_test,object,8091,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,details,float64,736,7355,0,4,0.0,23.63,2.57,69.68,4.99,4.99,4.99,9.99,499.99,17392.64,35.734,5.768,-185.41,232.67
5,device,object,8091,0,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,event_dt,datetime64[ns],6243,1848,0,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,event_name,object,6243,1848,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,first_date,datetime64[ns],8091,0,0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,group,object,8091,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,region,object,8091,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,user_id,object,8091,0,0,2841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Исследование горизонта анализа

Проверим на события в пределах лайтайма

In [59]:
# Рассчитать количество событий, которые не укладываются в лайфтайм (больше чем 14 дней)
events_outside_lifetime_count = len(participants_cleaned[(participants_cleaned['event_dt'] - 
                                                          participants_cleaned['first_date']).dt.days > 14])

print("Количество событий, не укладывающихся в лайфтайм (больше чем 14 дней):", events_outside_lifetime_count)

Количество событий, не укладывающихся в лайфтайм (больше чем 14 дней): 528


Все события, не укладывающиеся в лайфтайм, не являются единственными у пользователей, поэтому удаление строк с этими событиями никак не затронет наших пользователей

In [60]:
# Подсчитать и вывести количество строк до удаления
count_before = len(participants_cleaned)

# Удалить строки, в которых 'event_dt' больше чем 'first_date' + 14 дней, но оставить строки с NaN
participants_cleaned = participants_cleaned[participants_cleaned['event_dt'].le(participants_cleaned['first_date'] + pd.Timedelta(days=14)) | participants_cleaned['event_dt'].isna()]

# Пересортировать индексы, если необходимо
participants_cleaned.reset_index(drop=True, inplace=True)

# Подсчитать и вывести количество удаленных строк
count_deleted = count_before - len(participants_cleaned)

# Вывести количество удаленных строк
print("Количество удаленных строк:", count_deleted)


Количество удаленных строк: 528


In [61]:
dataframe_statistics(participants_cleaned)

,name,dtype,count,na_count,zero_count,unique_count,neg_count,mean,se_mean,sigma,min,25%,50%,75%,max,sum,kurtosis-3,adj_skewness,mean_-3sigma,mean_+3sigma
2,ab_test,object,7563,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,details,float64,667,6896,0,4,0.0,23.91,2.72,70.24,4.99,4.99,4.99,9.99,499.99,15948.33,35.11,5.723,-186.81,234.63
5,device,object,7563,0,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,event_dt,datetime64[ns],5715,1848,0,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,event_name,object,5715,1848,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,first_date,datetime64[ns],7563,0,0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,group,object,7563,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,region,object,7563,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,user_id,object,7563,0,0,2841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
latest_event_date = participants_cleaned['event_dt'].max()
print("Самая поздняя дата события:", latest_event_date)

Самая поздняя дата события: 2020-12-27 00:00:00


Создадим таблицу пользователей с количеством совершенных событий каждого типа

In [63]:
# Создание сводной таблицы
events_user = participants_cleaned.pivot_table(index='user_id', columns='event_name', values='event_dt', aggfunc='count', fill_value=0)

# Переименование столбцов для ясности
events_user.columns = [f'{col}_count' for col in events_user.columns]

# Сбросим индекс для получения user_id как столбца
events_user = events_user.reset_index()

# Вывод первых строк сводной таблицы
display(events_user.head())


,user_id,login_count,product_cart_count,product_page_count,purchase_count
0,00EFA157F7B6E1C4,3,3,3,0
1,015CCC27BDB640E1,2,0,2,0
2,016F758EB5C5A5DA,3,0,0,0
3,020A95B66F363AFB,3,0,3,0
4,021E3EC8A37EE2E3,2,0,0,2


Создадим таблицу с датами первых событий каждого типа

In [64]:
first_dates = participants_cleaned.pivot_table(index='user_id', columns='event_name', values='event_dt', aggfunc='min')

# Переименовываем столбцы
first_dates.columns = [f'{col}_first_date' for col in first_dates.columns]

# Сбрасываем индекс, чтобы получить user_id в виде столбца
first_dates = first_dates.reset_index()

# Выводим первые строки таблицы
display(first_dates.head())

,user_id,login_first_date,product_cart_first_date,product_page_first_date,purchase_first_date
0,00EFA157F7B6E1C4,2020-12-09,2020-12-09,2020-12-09,NaT
1,015CCC27BDB640E1,2020-12-09,NaT,2020-12-09,NaT
2,016F758EB5C5A5DA,2020-12-13,NaT,NaT,NaT
3,020A95B66F363AFB,2020-12-07,NaT,2020-12-07,NaT
4,021E3EC8A37EE2E3,2020-12-07,NaT,NaT,2020-12-07


In [65]:
# Преобразовываем сводную таблицу в обычный датафрейм
first_dates_df = first_dates.reset_index()

Посчитаем прошедшее время между регистрацией и совершением события каждого типа

In [66]:
# Создаем таблицу с уникальными пользователями и их датами регистрации
unique_users = participants_cleaned.groupby('user_id')['first_date'].min().reset_index()

# Переименовываем столбец с датой регистрации для удобства
unique_users.rename(columns={'first_date': 'registration_date'}, inplace=True)

# Объединяем таблицы по столбцу 'user_id'
first_dates_df = first_dates_df.merge(unique_users, on='user_id', how='left')

In [67]:
# Выбираем столбцы для вычисления разницы
columns_to_calculate = ['login_first_date', 'product_cart_first_date', 'product_page_first_date', 'purchase_first_date']

# Вычисляем разницу и сохраняем в новых столбцах
for col in columns_to_calculate:
    new_col_name = f'days_from_registration_to_{col}'
    first_dates_df[new_col_name] = first_dates_df[col] - first_dates_df['registration_date']

# Выводим результат
display(first_dates_df.head())


,index,user_id,login_first_date,product_cart_first_date,product_page_first_date,purchase_first_date,registration_date,days_from_registration_to_login_first_date,days_from_registration_to_product_cart_first_date,days_from_registration_to_product_page_first_date,days_from_registration_to_purchase_first_date
0,0,00EFA157F7B6E1C4,2020-12-09,2020-12-09,2020-12-09,NaT,2020-12-09,0 days,0 days,0 days,NaT
1,1,015CCC27BDB640E1,2020-12-09,NaT,2020-12-09,NaT,2020-12-09,0 days,NaT,0 days,NaT
2,2,016F758EB5C5A5DA,2020-12-13,NaT,NaT,NaT,2020-12-13,0 days,NaT,NaT,NaT
3,3,020A95B66F363AFB,2020-12-07,NaT,2020-12-07,NaT,2020-12-07,0 days,NaT,0 days,NaT
4,4,021E3EC8A37EE2E3,2020-12-07,NaT,NaT,2020-12-07,2020-12-07,0 days,NaT,NaT,0 days


In [68]:
# Рассчитываем медиану и среднее для каждой из переменных
medians = first_dates_df[['days_from_registration_to_login_first_date',
                         'days_from_registration_to_product_cart_first_date',
                         'days_from_registration_to_product_page_first_date',
                         'days_from_registration_to_purchase_first_date']].median()

means = first_dates_df[['days_from_registration_to_login_first_date',
                       'days_from_registration_to_product_cart_first_date',
                       'days_from_registration_to_product_page_first_date',
                       'days_from_registration_to_purchase_first_date']].mean()

# Выводим результат
print("Медианы:")
print(medians)
print("\nСредние значения:")
print(means)


Медианы:
days_from_registration_to_login_first_date          0 days
days_from_registration_to_product_cart_first_date   0 days
days_from_registration_to_product_page_first_date   0 days
days_from_registration_to_purchase_first_date       0 days
dtype: timedelta64[ns]

Средние значения:
days_from_registration_to_login_first_date          0 days 01:28:32.903225806
days_from_registration_to_product_cart_first_date   0 days 01:21:17.419354838
days_from_registration_to_product_page_first_date   0 days 01:36:29.690721649
days_from_registration_to_purchase_first_date          0 days 01:01:52.500000
dtype: timedelta64[ns]


In [69]:
df = first_dates_df

df['days_from_registration_to_login_first_date'] = df['days_from_registration_to_login_first_date'].apply(lambda x: x.total_seconds() / (24 * 3600))
df['days_from_registration_to_product_cart_first_date'] = df['days_from_registration_to_product_cart_first_date'].apply(lambda x: x.total_seconds() / (24 * 3600))
df['days_from_registration_to_product_page_first_date'] = df['days_from_registration_to_product_page_first_date'].apply(lambda x: x.total_seconds() / (24 * 3600))
df['days_from_registration_to_purchase_first_date'] = df['days_from_registration_to_purchase_first_date'].apply(lambda x: x.total_seconds() / (24 * 3600))


Выведем распределение первого события каждого типа по лайфтайму

In [70]:
df = first_dates_df
variables = [
    'days_from_registration_to_login_first_date',
    'days_from_registration_to_product_cart_first_date',
    'days_from_registration_to_product_page_first_date',
    'days_from_registration_to_purchase_first_date'
]

titles = [
    'Гистограмма распределения первого логина по лайфтайму',
    'Гистограмма распределения первой корзины по лайфтайму',
    'Гистограмма распределения первой страницы товара по лайфтайму',
    'Гистограмма распределения первой покупки по лайфтайму'
]

for variable, title in zip(variables, titles):
    fig = px.histogram(df, x=variable, nbins=10, title=title, histnorm='percent')
    fig.update_xaxes(title_text='Количество дней')
    fig.update_yaxes(title_text='Проценты')
    fig.show()


Рассчитаем среднее количество регистрирующихся пользователей в день

In [71]:
# Сгруппируем данные по столбцу 'first_date' и подсчитаем количество уникальных пользователей (регистраций) в каждую дату
registrations_per_day = participants_cleaned.groupby('first_date')['user_id'].nunique()

# Рассчитайте среднее количество регистрирующихся пользователей в день
mean_registrations_per_day = registrations_per_day.mean()

print(mean_registrations_per_day)


405.85714285714283


В датасете практически все первые события происходят в день регистрации

### Вывод о соответствии

ТЗ было нарушено многократно во всех возможных точках. 

1. 1602 пользователя оказались задействованы в разных тестах конкурирующих тестах, в результате чего становится не вполне понятно как у них исследовать экспериментальный эффект. 
2. В то время как регистрация должна была закончится 21 декабря, она длилась до 23 декабря. Пользователи должны были совершать события вплоть до 4 января, но последним днем совершения события было 30 декабря. Как потом стало очевидно, оба этих факта оказались полностью нивелированы разрушением экспериментальной схемы :) Однако 1002 пользователя пришлось удалить
3. Не все пользователи оказались из целевого региона. Способ определения целевого региона не вполне понятен, но так или иначе, 350 пользователей было удалено.
4. В течение всего периода регистрации в группу Б набиралось значимо меньше пользователей. 
5. В силу не до конца понятных причин, в датасете мы наблюдаем серьезную аномалию: единственной когортой пользователей которые в 100% случаев совершали события являлись пользователи группа А, зарегистрированные, начиная с 14 декабря. Все остальные когорты пользователей содержат высокий процент тех кто не совершил ни одного действия. В результате пришлось удалить всех пользователей, зарегистрированных с 14 декабря, поскольку они оказались в заведомо неравных условиях и сравнение их по выборкам потеряло какой-либо смысл.
6. В соответствие с ТЗ все события должны были совершаться пользователями в течение изучаемого лайфтайма, т.е. 14 дней с момента регистравии. Тем не менее, 528 событий оказалось в наших датасетах за пределами 14-дневных лайфтаймов, их пришлось удалить. В итоге, последним днем совершенных событий оказалось 27 декабря, что формирует идеальный cut-off point для исследования с регистрацией с 7 по 13 декабря и 14 дневным лайфтаймом :) При этом, следует отметить, что пользователи, сумевшие совершить события, в основном совершали их в первые же сутки с момента регистрации.
7. В ячейках ниже находятся скрины из калькуляторов, показывающие, что вообще говоря, для того чтобы достоверно обнаружить сдвиг конверсии в 5% при базовой конверсии 50%, нужен отнюдь не 2841 пользователь, а примерно в 4.5 раза больше, что вполне достижимо с такими же темпами экспозиции за месяц. При этом данные цифры учитывают лишь тех кто так или иначе попал в воронку, а в нашу воронку попали лишь 993 пользователя, т.е. среднее количество регистраций в группы тесты не превышало 142 человек в сутки.

![Описание изображения](https://dl.dropboxusercontent.com/s/mo9tgt2xk83qtrsu7pr73/3618.png?rlkey=o1tmn1cm51ccyhvwiusydpimm&dl=0)

![Описание изображения](https://dl.dropboxusercontent.com/s/s36cxx0031m6a7ljx1jyl/3615.png?rlkey=snu41fbtd6j2skudxinn2sbiz&dl=0)

## Исследовательский анализ данных

Изучим распределение количества событий на пользователя в разрезе групп
теста

In [72]:
# Создаем таблицу с уникальными пользователями и их группами
unique_users_groups = participants_cleaned[['user_id', 'group']].drop_duplicates()

# Объединяем таблицы по столбцу 'user_id' для добавления группы к каждому уникальному пользователю
events_user = events_user.merge(unique_users_groups, on='user_id', how='left')


In [73]:
events_user.head()

,user_id,login_count,product_cart_count,product_page_count,purchase_count,group
0,00EFA157F7B6E1C4,3,3,3,0,A
1,015CCC27BDB640E1,2,0,2,0,A
2,016F758EB5C5A5DA,3,0,0,0,A
3,020A95B66F363AFB,3,0,3,0,B
4,021E3EC8A37EE2E3,2,0,0,2,A


Выведем число событий на среднего пользователя по группам

In [74]:
# Список переменных
variables = ['login_count', 'product_cart_count', 'product_page_count', 'purchase_count']

# Создаем пустой DataFrame для хранения результатов
results2 = pd.DataFrame()

# Цикл для расчета средних значений для каждой переменной
for variable in variables:
    average_data = events_user.groupby('group')[variable].mean()
    results2[variable] = average_data

# Транспонируем таблицу
results2 = results2.T

In [75]:
results2 = results2.reset_index()

In [76]:
results2['Difference'] = results2['A']-results2['B']
display(results2)

group,index,A,B,Difference
0,login_count,2.811966,2.786765,0.025201
1,product_cart_count,0.724786,0.671569,0.053218
2,product_page_count,1.613675,1.529412,0.084263
3,purchase_count,0.629060,0.732843,-0.103783


Количество событий каждого типа на среднего пользователя в группах А и B принципиально не отличается.

In [77]:
def plot_bar_chart(data, variable_name):
    x_label = variable_name
    y_label = 'Количество пользователей'
    title = f'Распределение {variable_name} с разбивкой по группам'

    fig = px.bar(
        data,
        x=variable_name,
        y='count',  # Название переменной, отражающей количество пользователей
        color='group',
        labels={variable_name: x_label, 'count': y_label},
    )

    fig.update_layout(
        title=title,
        xaxis=dict(title=x_label),
        yaxis=dict(title=y_label),
        xaxis_type='category'
    )

    fig.show()

# Создаем графики для всех четырех переменных
variable_names = ['login_count', 'product_cart_count', 'product_page_count', 'purchase_count']

for variable_name in variable_names:
    data = events_user.groupby(['group', variable_name]).size().reset_index(name='count')
    plot_bar_chart(data, variable_name)


In [78]:
variables = ['login_count', 'product_cart_count', 'product_page_count', 'purchase_count']

for variable in variables:
    # Рассчитываем процентное соотношение клиентов
    percentage_data = (events_user.groupby([variable, 'group']).size() / events_user.groupby(variable).size()).reset_index()
    percentage_data.columns = [variable, 'group', 'percentage']

    # Создаем диаграмму
    fig = px.bar(percentage_data, x=variable, y='percentage', color='group',
                 title=f'Процентное соотношение клиентов по {variable} с учетом группы',
                 labels={variable: f'Количество {variable}', 'percentage': 'Доля', 'group': 'Группа'})

    # Отобразим график
    fig.show()

В разбиении распределений по группам ничего критичного не проглядывается с учетом абсолютных значений.

Изучим динамику количества событий в группах теста по дням

In [79]:
# Сгруппируйте данные по дате события и группе
grouped_data = participants_cleaned.groupby(['event_dt', 'group']).size().reset_index(name='event_count')

# Создайте график с использованием Plotly
fig = go.Figure()

# Добавьте линии для каждой группы
for group in grouped_data['group'].unique():
    data = grouped_data[grouped_data['group'] == group]
    fig.add_trace(go.Scatter(x=data['event_dt'], y=data['event_count'], mode='lines', name=group))

# Задайте область для закраски
fig.add_shape(
    type='rect',
    x0='2020-12-25',
    x1=max(grouped_data['event_dt']),
    y0=0,
    y1=max(grouped_data['event_count']),
    line=dict(color='yellow', width=2),  # Используем желтый цвет
    fillcolor='rgba(255, 255, 0, 0.2)'  # Используем желтую заливку
)

# Настройте метки и заголовок
fig.update_layout(
    xaxis_title='Дата',
    yaxis_title='Количество событий',
    title='Динамика количества событий в группах теста с выделением с 20 декабря 2020 года'
)

# Покажите график
fig.show()



C:\Users\Sergey\anaconda3\envs\da_practicum_env\lib\site-packages\_plotly_utils\basevalidators.py:107: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Динамика числа событий в лайфтайме по группам выглядит согласованно. Пользователи из группы А со 8 декабря по 26 декабря совершают больше событий.

In [80]:
# Сгруппируйте данные по дате события и группе
grouped_data = participants_cleaned.groupby(['event_dt', 'group']).size().reset_index(name='event_count')

# Рассчитайте общее количество событий для каждой даты
total_events_per_date = grouped_data.groupby('event_dt')['event_count'].sum().reset_index()

# Объедините данные, чтобы рассчитать процентное соотношение событий для каждой группы
grouped_data = grouped_data.merge(total_events_per_date, on='event_dt', suffixes=('', '_total'))
grouped_data['event_percent'] = (grouped_data['event_count'] / grouped_data['event_count_total']) * 100

# Создайте график динамики процентного соотношения событий
fig = px.bar(grouped_data, x='event_dt', y='event_percent', color='group',
             labels={'event_percent': 'Процент событий', 'event_dt': 'Дата'},
             title='Процентное соотношение событий в группах теста')

# Добавьте область для выделения начиная с 20 декабря 2020 года
fig.add_shape(
    type='rect',
    x0='2020-12-25',
    x1=grouped_data['event_dt'].max(),
    y0=0,
    y1=100,
    fillcolor='green',  # Зеленый цвет
    opacity=0.7,
    layer='below'
)

fig.show()


C:\Users\Sergey\anaconda3\envs\da_practicum_env\lib\site-packages\_plotly_utils\basevalidators.py:107: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Ничего критичного в распределении числа событий по датам мы не обнаруживаем. Это число уверенно уходит в ноль к 27 декабря, начиная с 300 с лишним 7 декабря. 

Проверим не совпадает ли наш тест по времени с маркетинговыми событиями.

In [81]:
# Найти самую раннюю и самую позднюю дату из столбца event_dt
earliest_date = participants_cleaned['event_dt'].min()
latest_date = participants_cleaned['event_dt'].max()

# Задать test_start_date и test_end_date
test_start_date = earliest_date
test_end_date = latest_date

# Найти моду колонки region в датасете participants_cleaned
mode_region = participants_cleaned['region'].mode()[0]

# Проверить пересечение временных интервалов теста и маркетинговых событий
intersecting_events = marketing_events[
    ((marketing_events['start_dt'] <= test_end_date) & (marketing_events['finish_dt'] >= test_start_date))
    & marketing_events['regions'].str.contains(mode_region)
]

if intersecting_events.empty:
    print("Нет пересечений между временем проведения теста и маркетинговыми событиями в регионе теста")
else:
    print("Есть пересечения между временем проведения теста и следующими маркетинговыми событиями в регионе теста:")
    print(intersecting_events)


Есть пересечения между временем проведения теста и следующими маркетинговыми событиями в регионе теста:
                       name        regions   start_dt  finish_dt
0  Christmas&New Year Promo  EU, N.America 2020-12-25 2021-01-03


Мы видим, что лишь одно событие пересекается с датами в которые пользователи совершали события. При этом, никакого ощутимого влияния на совершение пользователями событий, судя по их количеству, это не возымело: с одной стороны, именно с 25 декабря динамика между группами была не просто согласованной, а выровненной. С другой стороны, она становилась все более согласованной, начиная с 12 декабря, а выровнялась уже на последнем 3-х дневном отрезке в ассимптотической области графика: в нуле количество событий было бы вообще математически равно. В общем, у нас недостаточно данных чтобы судить о том, может ли повлиять данное событие на результаты теста: в нашем случае оно если и повлияло, то на результатах это никак особенно не отразилось. Мы действительно видим "ощутимый отскок" в группе Б с 5 событий 25 декабря до 9 событий 26 декабря, но в абсолютных значениях это окажется в пределах погрешности.

Исследуем продуктовые воронки

In [82]:
# Подсчет количества пользователей, начавших каждый этап воронки
funnel_data = participants_cleaned.groupby(['group', 'event_name']).agg({'user_id': 'nunique'}).reset_index()

# Определите порядок событий, который вы хотите использовать
custom_event_order = ['login', 'product_page', 'product_cart', 'purchase']

# Создание сводной таблицы для построения воронки
funnel_pivot = funnel_data.pivot(index='event_name', columns='group', values='user_id')

# Установка порядка событий в соответствии с custom_event_order
funnel_pivot = funnel_pivot.loc[custom_event_order]

# Расчет доли пользователей, перешедших с одного этапа на следующий
funnel_pivot['group_A_conversion'] = funnel_pivot['A'] / funnel_pivot['A'].iloc[0]
funnel_pivot['group_B_conversion'] = funnel_pivot['B'] / funnel_pivot['B'].iloc[0]

# Вывод воронки
display(funnel_pivot)


group,A,B,group_A_conversion,group_B_conversion
event_name,,,,
login,585,407,1.000000,1.000000
product_page,349,233,0.596581,0.572482
product_cart,150,98,0.256410,0.240786
purchase,142,114,0.242735,0.280098


In [83]:
# Создаем данные для воронки
event_names = ['login', 'product_page', 'product_cart', 'purchase']
user_counts_group_A = [participants_cleaned[(participants_cleaned['group'] == 'A') & (participants_cleaned['event_name'] == event_name)]['user_id'].nunique() for event_name in event_names]
user_counts_group_B = [participants_cleaned[(participants_cleaned['group'] == 'B') & (participants_cleaned['event_name'] == event_name)]['user_id'].nunique() for event_name in event_names]

# Создаем график продуктовой воронки для группы A
fig_group_A = go.Figure(go.Funnel(
    y=event_names,
    x=user_counts_group_A,
    textinfo="value+percent initial",
    name="Группа A"
))
fig_group_A.update_layout(title="Продуктовая воронка для группы A")

# Создаем график продуктовой воронки для группы B
fig_group_B = go.Figure(go.Funnel(
    y=event_names,
    x=user_counts_group_B,
    textinfo="value+percent initial",
    name="Группа B"
))
fig_group_B.update_layout(title="Продуктовая воронка для группы B")

# Отображаем графики продуктовой воронки
fig_group_A.show()
fig_group_B.show()

Судя по воронке, процент тех кто зашел на страницу продукта и на страницу корзины в группе Б ниже чем в группе А. Тем не менее, в группе Б 28% тех кто совершил покупку от тех кто сумел авторизоваться на странице, а в группе А всего 24%. Значимость подобных различий сомнительна с учетом размера выборок, но мы это скоро установим. Характерной особенностью группы Б в данном случае является то что произошел прирост именно в области перехода с корзины к покупке, т.е. этот выигрыш группа Б получила за счет удобной реализации быстрой покупки в обход формирования корзины заказа.

Выведем для справки статистику по пользователям совершавшим покупки:

In [84]:
# Фильтрация данных по группам A и B
group_A = participants_cleaned[participants_cleaned['group'] == 'A']
group_B = participants_cleaned[participants_cleaned['group'] == 'B']

# Фильтрация данных только для пользователей, совершивших покупки
group_A_purchases = group_A[group_A['event_name'] == 'purchase']
group_B_purchases = group_B[group_B['event_name'] == 'purchase']

# Сумма потраченных денег
total_spent_A = group_A_purchases['details'].sum()
total_spent_B = group_B_purchases['details'].sum()

# Общее число покупок
total_purchases_A = len(group_A_purchases)
total_purchases_B = len(group_B_purchases)

# Среднее число покупок на пользователя
average_purchases_per_user_A = round(total_purchases_A / group_A_purchases['user_id'].nunique(), 2)
average_purchases_per_user_B = round(total_purchases_B / group_B_purchases['user_id'].nunique(), 2)

# Средняя сумма покупки
average_purchase_amount_A = round(total_spent_A / total_purchases_A, 2) if total_purchases_A > 0 else 0
average_purchase_amount_B = round(total_spent_B / total_purchases_B, 2) if total_purchases_B > 0 else 0

# Средняя потраченная сумма на пользователя
average_spent_per_user_A = round(total_spent_A / group_A_purchases['user_id'].nunique(), 2)
average_spent_per_user_B = round(total_spent_B / group_B_purchases['user_id'].nunique(), 2)

# Создание таблицы
data = {
    'Группа': ['A', 'B'],
    'Сумма потраченных денег': [round(total_spent_A, 2), round(total_spent_B, 2)],
    'Общее число покупок': [total_purchases_A, total_purchases_B],
    'Среднее число покупок на пользователя': [average_purchases_per_user_A, average_purchases_per_user_B],
    'Средняя сумма покупки': [average_purchase_amount_A, average_purchase_amount_B],
    'Средняя потраченная сумма на пользователя': [average_spent_per_user_A, average_spent_per_user_B]
}

df = pd.DataFrame(data)

# Вывод таблицы
display(df)



,Группа,Сумма потраченных денег,Общее число покупок,Среднее число покупок на пользователя,Средняя сумма покупки,Средняя потраченная сумма на пользователя
0,A,8956.32,368,2.59,24.34,63.07
1,B,6992.01,299,2.62,23.38,61.33


Как мы видим, в среднем пользователь из группы Б совершал чуть больше покупок, но средняя сумма покупки была чуть ниже и средняя сумма которую тратил пользовател группы Б тоже чуть ниже.

## Анализ АБ теста

H0: Внедрение улучшенной рекомендательной системы никак не повлияет на покупателей, так что значимых различий по доле уникальных пользователей, совершивших соответствующее событие, от общего числа пользователей в группе наблюдаться не будет между контрольной выборкой А и экспериментальной выборкой Б.

H1: Внедрение улучшенной рекомендательной системы окажет влияние на конверсию для соответствующих переменных. Конверсия в группе Б окажется значимо выше.

In [92]:
def analyze_conversion(data, event_names):
    for event_name in event_names:
        # Выделите пользователей каждой группы
        group_A = data[data['group'] == 'A']
        group_B = data[data['group'] == 'B']

        # Определите количество уникальных пользователей в каждой группе
        total_users_A = group_A['user_id'].nunique()
        total_users_B = group_B['user_id'].nunique()

        # Подсчитайте количество пользователей в каждой группе, выполнивших указанное событие
        event_users_A = group_A[group_A['event_name'] == event_name]['user_id'].nunique()
        event_users_B = group_B[group_B['event_name'] == event_name]['user_id'].nunique()

        # Рассчитайте конверсию для каждой группы
        conversion_A = event_users_A / total_users_A
        conversion_B = event_users_B / total_users_B
        
        print(total_users_A, total_users_B)

        # Выведите результаты
        print(f'Анализ конверсии для события "{event_name}":')
        print(f'Количество пользователей в группе A: {total_users_A}')
        print(f'Количество пользователей в группе B: {total_users_B}')
        print(f'Количество пользователей в группе A, выполнивших событие "{event_name}": {event_users_A}')
        print(f'Количество пользователей в группе B, выполнивших событие "{event_name}": {event_users_B}')
        print(f'Конверсия в группе A: {conversion_A:.3f}')
        print(f'Конверсия в группе B: {conversion_B:.3f}')

        # Выполнение двухпропорционального Z-теста
        count = np.array([event_users_A, event_users_B])
        nobs = np.array([total_users_A, total_users_B])
        z_stat, p_value = proportions_ztest(count, nobs, alternative='smaller')

        # Вывод результатов теста
        print(f'Z-statistic = {z_stat:.3f}; p = {p_value:.3f}')
        print('\n')

# Пример использования функции для анализа конверсии по всем четырем событиям
event_names = ['login', 'product_page', 'product_cart', 'purchase']
analyze_conversion(participants_cleaned, event_names)


1615 1226
Анализ конверсии для события "login":
Количество пользователей в группе A: 1615
Количество пользователей в группе B: 1226
Количество пользователей в группе A, выполнивших событие "login": 585
Количество пользователей в группе B, выполнивших событие "login": 407
Конверсия в группе A: 0.362
Конверсия в группе B: 0.332
Z-statistic = 1.675; p = 0.953


1615 1226
Анализ конверсии для события "product_page":
Количество пользователей в группе A: 1615
Количество пользователей в группе B: 1226
Количество пользователей в группе A, выполнивших событие "product_page": 349
Количество пользователей в группе B, выполнивших событие "product_page": 233
Конверсия в группе A: 0.216
Конверсия в группе B: 0.190
Z-statistic = 1.704; p = 0.956


1615 1226
Анализ конверсии для события "product_cart":
Количество пользователей в группе A: 1615
Количество пользователей в группе B: 1226
Количество пользователей в группе A, выполнивших событие "product_cart": 150
Количество пользователей в группе B, выпо

Значимых различий между выборками не обнаружено, отвергнуть гипотезу H0 не удалось.

H0: Внедрение улучшенной рекомендательной системы никак не повлияет на покупателей, так что значимых различий по доле уникальных пользователей, совершивших соответствующее событие, от общего числа пользователей в группе попавших в продуктовую воронку  наблюдаться не будет между контрольной выборкой А и экспериментальной выборкой Б.

H1: Внедрение улучшенной рекомендательной системы окажет влияние на конверсию для соответствующих переменных. Конверсия в группе Б окажется значимо выше.

In [93]:
def analyze_conversion(data, event_names):
    for event_name in event_names:
        # Выделите пользователей каждой группы
        group_A = data[data['group'] == 'A']
        group_B = data[data['group'] == 'B']

        # Определите количество уникальных пользователей в каждой группе
        total_users_A = group_A['user_id'].nunique()
        total_users_B = group_B['user_id'].nunique()

        # Подсчитайте количество пользователей в каждой группе, выполнивших указанное событие
        event_users_A = group_A[group_A['event_name'] == event_name]['user_id'].nunique()
        event_users_B = group_B[group_B['event_name'] == event_name]['user_id'].nunique()

        # Подсчитайте количество пользователей, совершивших хотя бы одно событие
        users_with_events_A = group_A[group_A['event_name'].notnull()]['user_id'].nunique()
        users_with_events_B = group_B[group_B['event_name'].notnull()]['user_id'].nunique()

        # Рассчитайте конверсию для каждой группы
        conversion_A = event_users_A / users_with_events_A
        conversion_B = event_users_B / users_with_events_B

        # Выведите результаты
        print(f'Анализ конверсии для события "{event_name}":')
        print(f'Количество пользователей в группе A: {users_with_events_A}')
        print(f'Количество пользователей в группе B: {users_with_events_B}')
        print(f'Количество пользователей в группе A, выполнивших событие "{event_name}": {event_users_A}')
        print(f'Количество пользователей в группе B, выполнивших событие "{event_name}": {event_users_B}')
        print(f'Конверсия в группе A: {conversion_A:.3f}')
        print(f'Конверсия в группе B: {conversion_B:.3f}')

        # Выполнение двухпропорционального Z-теста
        count = np.array([event_users_A, event_users_B])
        nobs = np.array([users_with_events_A, users_with_events_B])
        z_stat, p_value = proportions_ztest(count, nobs, alternative='smaller')

        # Вывод результатов теста
        print(f'Z-statistic = {z_stat:.3f}; p = {p_value:.3f}')
        print('\n')

# Пример использования функции для анализа конверсии по всем четырем событиям
event_names = ['login', 'product_page', 'product_cart', 'purchase']
analyze_conversion(participants_cleaned, event_names)



Анализ конверсии для события "login":
Количество пользователей в группе A: 585
Количество пользователей в группе B: 408
Количество пользователей в группе A, выполнивших событие "login": 585
Количество пользователей в группе B, выполнивших событие "login": 407
Конверсия в группе A: 1.000
Конверсия в группе B: 0.998
Z-statistic = 1.198; p = 0.885


Анализ конверсии для события "product_page":
Количество пользователей в группе A: 585
Количество пользователей в группе B: 408
Количество пользователей в группе A, выполнивших событие "product_page": 349
Количество пользователей в группе B, выполнивших событие "product_page": 233
Конверсия в группе A: 0.597
Конверсия в группе B: 0.571
Z-statistic = 0.803; p = 0.789


Анализ конверсии для события "product_cart":
Количество пользователей в группе A: 585
Количество пользователей в группе B: 408
Количество пользователей в группе A, выполнивших событие "product_cart": 150
Количество пользователей в группе B, выполнивших событие "product_cart": 98
К

Значимых различий между выборками не обнаружено, отвергнуть гипотезу H0 не удалось.

Т.о. мы получили полезный опыт работы с маленькими выборками. Проблема состоит в том, что мы в состоянии получить высокие значения критерия и это всегда приятно. Однако, статистическая значимость, к сожалению, зависит не только от собственного значения критерия, но и от объема выборки. Это интуитивно понятно: чем выборка меньше, тем более велик вклад i-го испытуемого в дисперсию показателя, а именно это, по факту, значение p-value и отражает. Мы наблюдали интересный финал, когда, действительно, в условиях нестрогой продуктовой воронки пользователи группы Б частично обошли стороной формирование корзины и просто купили то что хотели и, в итоге, конверсия в число покупок оказалась выше. Но различия оказались недостаточно велики для того чтобы стать значимыми. Поэтому если рассматривать конверсию на каждом этапе воронки, ожидаемый эффект применения улучшенной рекомендательной системы в исследовании не обнаружился. Так или иначе, по факту мы резюмируем: АБ тест не удался по причине недостатка в реализации изначального дизайна.

Почему мы не используем поправку на множественные сравнения. Важно понимать что эта проблема находится больше в философской плоскости чем в какой-либо еще. Дело в том, что вероятность ложнопозитива, т.е. ошибки первого рода, увеличивается не только с количеством замеров на конкретных выборках: она увеличивается в принципе в процессе работы. Если довести эту идею до абсурда, вероятность получения ложнопозитива для аналитика, работающего более 30 лет значимо выше чем для того, кто обсчитал свою первую матрицу. Исходя из этого надо понимать и как должны быть устроены способы борьбы с этой проблемой, они так или иначе сводятся к двум путям: к снижению пороговой альфы или к демонстрации воспроизводимости результатов с применением других критериев и/или в новых замерах. Формула для расчета вероятности ложнопозитива при множественных исследованиях интуитивно понятна: 1-(1-alpha)**m, где m - количество проведенных тестов. Распространенные поправки для альфы помогают при множественных тестированиях (типа множественных парных сравнений, ANOVA и пр): они помогают оценить пороговое значение альфы для удержания вероятности возникновения ложнопозитива на приемлемом уровне. К примеру, с этой целью мы можем поделить желаемый уровень значимости на количество проведенных сравнений (alpha/m) или воспользоваться более сложным алгоритмом расчета альфы. Так или иначе, надо помнить, что статистика это инструмент оценки, а не готовый ответ. Во многих случаях мы обречены на ложнопозитивы и тут ничего не поделать. Отчасти поэтому мы результаты, включающие в себя вероятность ошибки, превышающую 1%, даже не рассматриваем.

## Выводы

В результате исследования событий удалось установить следующее:
    1. Количество событий каждого типа на среднего пользователя в группах А и B принципиально не отличается.
    2. В разбиении распределений по группам ничего критичного не проглядывается с учетом абсолютных значений количества пользователей.
    3. Динамика числа событий в лайфтайме по группам выглядит согласованно.
    4. Лишь одно событие пересекается с датами в которые пользователи совершали события. При этом, никакого ощутимого влияния на совершение пользователями событий, судя по их количеству, это не возымело: с одной стороны, именно с 25 декабря динамика между группами была не просто согласованной, а выровненной. С другой стороны, она становилась все более согласованной, начиная с 12 декабря, а выровнялась уже на последнем 3-х дневном отрезке в ассимптотической области графика: в нуле количество событий было бы вообще математически равно. 
    5. Судя по воронке, конверсия заходов на страницу продукта и на страницу корзины в группе Б ниже чем в группе А. Тем не менее, в группе Б 28% тех кто совершил покупку от тех кто сумел авторизоваться на странице, а в группе А всего 24%. 
    6. В среднем пользователь из группы Б совершал чуть больше покупок, но средняя сумма покупки была чуть ниже и средняя сумма которую тратил пользовател группы Б тоже чуть ниже.
    
В качестве подтверждения нашей, довольно скептической оценки результатов исследования, мы сравнили конверсии в продуктовой воронке статистически и не нашли значимых различий. Связано это было с тем, что авторами были проигнорированы требования их собственного дизайна, в результате чего к финишу мы пришли с менее чем 1 тысячей сумевших авторизоваться уникальных пользователей на 2 выборки. Хочется думать, что те намеки на повышенную конверсию по продаже в группе Б на самом деле отражают реальный эффект улучшенной рекомендательной системы, но этого мы не знаем. 

Что же случилось?

В ходе исследования переменных нами было обнаружено, что:

1. 1602 пользователя оказались задействованы в конкурирующих тестах, в результате чего становится не вполне понятно как у них исследовать экспериментальный эффект.
2. В то время как регистрация должна была закончиться 21 декабря, она длилась до 23 декабря. Пользователи должны были совершать события вплоть до 4 января, но последним днем совершения события было 30 декабря. Как потом стало очевидно, оба этих факта оказались полностью нивелированы разрушением экспериментальной схемы :) Однако 1002 пользователя пришлось удалить
3. Не все пользователи оказались из целевого региона. Способ определения целевого региона не вполне понятен, но так или иначе, 350 пользователей было удалено.
4. В течение всего периода регистрации в группу Б набиралось значимо меньше пользователей.
5. В силу не до конца понятных причин, в датасете мы наблюдаем серьезную аномалию: единственной когортой пользователей которые в 100% случаев совершали события являлись пользователи группа А, зарегистрированные, начиная с 14 декабря. Все остальные когорты пользователей содержат высокий процент тех кто не совершил ни одного действия. В результате пришлось удалить всех пользователей, зарегистрированных с 14 декабря, поскольку они оказались в заведомо неравных условиях и сравнение их по выборкам потеряло какой-либо смысл.
6. В соответствие с ТЗ все события должны были совершаться пользователями в течение изучаемого лайфтайма, т.е. 14 дней с момента регистравии. Тем не менее, 528 событий оказалось в наших датасетах за пределами 14-дневных лайфтаймов, их пришлось удалить. В итоге, последним днем совершенных событий оказалось 27 декабря, что формирует идеальный cut-off point для исследования с регистрацией с 7 по 13 декабря и 14 дневным лайфтаймом :) При этом, следует отметить, что пользователи, сумевшие совершить события, в основном совершали их в первые же сутки с момента регистрации.
7. Согласно калькуляторам расчета объема выборки и времени, отводимого на набор пользователей, для того чтобы достоверно обнаружить сдвиг конверсии в 5% при базовой конверсии 50%, нужен отнюдь не 2841 пользователь, а примерно в 4.5 раза больше, что вполне достижимо с такими же темпами экспозиции за месяц. 

Так или иначе, мы не сумели обнаружить не только наличия значимых различий: мы вообще преимущества улучшенной рекомендательной системы на наших данных не заметили. Однако что явилось тому причиной, неудачно проведенный эксперимент или неудачное маркетинговое решение, к сожалению, нам осталось неведомо.


Рекомендации:

1. В работе с набором экспериментальных данных с выборке основным является точная разработка дизайна и безусловное следование намеченному плану. 
2. Если были сформулированы цели по процентам улучшения метрики, значит была возможность заранее составить представление о необходимых размерах выборок: тогда исследование не было бы запорото.
3. Факт непонятных проблем на этапе регистрация-авторизация, внезапно исчезнувших в одной из групп 14 декабря, говорит о недостаточном контроле угроз внутренней валидности.
4. Факт того, что нам пришлось ограничиться изучением лайфтаймов тех кто регистрировался в течение 7 дней, говорит о недостаточном контроле угроз внешней валидности: 993 пользователя это не сильно много, их в принципе можно было бы набрать и за сутки и потом мучаться в попытках объяснить как данные набранные конкретные 24 часа предлагается экстраполировать вообще на все время. Для того и изучаются лайфтаймы чтобы понять как выглядят распределения тех или иных показателей во времени.
5. На любом этапе можно было остановиться, перестать тратить время и деньги организации и перепланировать исследование. В науке широко распространены квазиэксперименты, созданные по принципу лоскутного одеяла. Причин у этого много, но в науке у ученых есть опция годами заниматься исследованием одной и той же проблемы. В бизнесе такой опции не существует: нужно быстро и точно дать ответы на поставленные вопросы, что можно сделать лишь если спланировать и провести исследование безошибочно с самого начала.


